In [ ]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
from train import FastflowTrainer
from omegaconf import OmegaConf
import numpy as np
import torch
from tqdm import tqdm
from data import FeaturesTrainDataset, FeaturesDatasetOOD

In [ ]:
cfg_path = "/workspaces/ood/fastflow/config.yaml"
cfg = OmegaConf.load(cfg_path)

In [ ]:
train_size = 100
train_dataset = FeaturesTrainDataset(
    "/workspaces/ood/data/feature_maps/cifar10train_wide_resnet50_layer2_layer3_layer4_1000_img_per_class.npy",
    ["layer2", "layer3", "layer4"],
    [(train_size, 512, 28, 28), (train_size, 1024, 14, 14), (train_size, 2048, 7, 7)],
)

In [ ]:
test_dataset = FeaturesDatasetOOD(
    "/workspaces/ood/data/feature_maps/cifar10test_wide_resnet50_layer2_layer3_layer4_100_img_per_class.npy",
    "/workspaces/ood/data/feature_maps/svhntest_wide_resnet50_layer2_layer3_layer4_100_img_per_class.npy",
    ["layer2", "layer3", "layer4"],
    [(1000, 512, 28, 28), (1000, 1024, 14, 14), (1000, 2048, 7, 7)],
)

In [ ]:
train_dataloader = DataLoader(
    train_dataset, batch_size=cfg.trainer.batch_size, shuffle=True
)
test_dataloader = DataLoader(
    test_dataset, batch_size=cfg.trainer.batch_size, shuffle=False
)

In [ ]:
trainer = FastflowTrainer(cfg)

trainer.train(train_dataloader, test_dataloader)

In [ ]:
# torch.save(trainer.model.state_dict(), f"debug_model.pth")

In [ ]:
# trainer.model.state_dict()

In [ ]:
# trainer = FastflowTrainer(cfg)
# trainer.model.load_state_dict(torch.load("debug_model.pth"))

In [ ]:
# from model import FastflowModel, get_logp


# def make_predictions(model, dataset, device):
#     """
#     returns logits for each samlple in dataset
#     """
#     model.eval()
#     preds_in = []
#     preds_out = []

#     for emb, label in tqdm(dataset):
#         avg_loss = 0
#         for layer_idx, encoder_activations in enumerate(emb):
#             encoder_activations = torch.unsqueeze(
#                 torch.tensor(encoder_activations).to(device), dim=0
#             )
#             (
#                 _,
#                 dim_feature_vector,
#                 im_height,
#                 im_width,
#             ) = encoder_activations.size()
#             decoder = model.decoders[layer_idx].to(device)
#             p_u, log_jac_det = decoder(encoder_activations)
#             print(log_jac_det, flush=True)
#             return
#             decoder_log_prob = get_logp(dim_feature_vector, p_u, log_jac_det)
#             avg_loss += decoder_log_prob.sum().detach().cpu().numpy()
#             print(p_u)
#             return
#         if label == 0:
#             preds_in.append(avg_loss)
#         elif label == 1:
#             preds_out.append(avg_loss)
#         else:
#             raise ValueError
#         print(preds_in, preds_out)
#         return

#     return preds_in, preds_out


# preds = make_predictions(trainer.model, test_dataset, torch.device("cuda"))